In [15]:
import re

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc"><h2>--- Day 3: Gear Ratios ---</h2><p>You and the Elf eventually reach a <a href="https://en.wikipedia.org/wiki/Gondola_lift" target="_blank">gondola lift</a> station; he says the gondola lift will take you up to the <em>water source</em>, but this is as far as he can bring you. You go inside.</p>
<p>It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.</p>
<p>"Aaah!"</p>
<p>You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.</p>
<p>The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can <em>add up all the part numbers</em> in the engine schematic, it should be easy to work out which part is missing.</p>
<p>The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently <em>any number adjacent to a symbol</em>, even diagonally, is a "part number" and should be included in your sum. (Periods (<code>.</code>) do not count as a symbol.)</p>
<p>Here is an example engine schematic:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, two numbers are <em>not</em> part numbers because they are not adjacent to a symbol: <code>114</code> (top right) and <code>58</code> (middle right). Every other number is adjacent to a symbol and so <em>is</em> a part number; their sum is <code><em>4361</em></code>.</p>
<p>Of course, the actual engine schematic is much larger. <em>What is the sum of all of the part numbers in the engine schematic?</em></p>
</article>


In [16]:
schema_test = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

print(schema_test)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


In [17]:
from re import Match


def get_neighbors(rows: int, cols: int, row: int, m: Match):
    start, end = m.span()
    neighbors = []
    if start - 1 > -1:
        neighbors.append((row, start - 1))

    if row - 1 > -1:
        neighbors.extend(
            (row - 1, col) for col in range(start - 1, end + 1) if 0 <= col < cols
        )

    if end + 1 < cols:
        neighbors.append((row, end))

    if row + 1 < rows:
        neighbors.extend(
            (row + 1, col) for col in range(end, start - 2, -1) if 0 <= col < cols
        )

    return neighbors


def isSymbol(s: str) -> bool:
    return not s.isdigit() and s != "."


def sum_part_numbers(s: str) -> int:
    count = 0
    lines = s.splitlines()
    rows, cols = len(lines), len(lines[0])

    for row in range(rows):
        for m in re.finditer(r"\d+", lines[row]):
            neighbors = get_neighbors(rows, cols, row, m)

            if any(isSymbol(lines[r][c]) for r, c in neighbors):
                count += int(m.group())
    return count


sum_part_numbers(schema_test), 4361

(4361, 4361)

In [18]:
with open("../input/day3.txt") as f:
    print(sum_part_numbers(f.read()))

529618


<link href="style.css" rel="stylesheet"></link>
<p>Your puzzle answer was <code>529618</code>.</p>
<p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.</p>
<p>You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.</p>
<p>Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.</p>
<p>The missing part wasn't the only issue - one of the gears in the engine is wrong. A <em>gear</em> is any <code>*</code> symbol that is adjacent to <em>exactly two part numbers</em>. Its <em>gear ratio</em> is the result of <span title="They're magic gears.">multiplying</span> those two numbers together.</p>
<p>This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.</p>
<p>Consider the same engine schematic again:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, there are <em>two</em> gears. The first is in the top left; it has part numbers <code>467</code> and <code>35</code>, so its gear ratio is <code>16345</code>. The second gear is in the lower right; its gear ratio is <code>451490</code>. (The <code>*</code> adjacent to <code>617</code> is <em>not</em> a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces <code><em>467835</em></code>.</p>
<p><em>What is the sum of all of the gear ratios in your engine schematic?</em></p>
</article>


In [19]:
schema_test = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

print(schema_test)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


In [20]:
from re import Match
from collections import defaultdict
from math import prod


def get_gear_neighbors(
    lines: list[int], rows: int, cols: int, row: int, m: Match, gears
) -> None:
    start, end = m.span()
    part = int(m.group())
    if start - 1 > -1 and lines[row][start - 1] == "*":
        gears[(row, start - 1)].append(part)

    if row - 1 > -1:
        for col in range(start - 1, end + 1):
            if 0 <= col < cols and lines[row - 1][col] == "*":
                gears[(row - 1, col)].append(part)

    if end + 1 < cols and lines[row][end] == "*":
        gears[(row, end)].append(part)

    if row + 1 < rows:
        for col in range(end, start - 2, -1):
            if 0 <= col < cols and lines[row + 1][col] == "*":
                gears[(row + 1, col)].append(part)


def sum_ratios(s: str) -> int:
    lines = s.splitlines()
    rows, cols = len(lines), len(lines[0])
    gears = defaultdict(list[int, int])
    for row in range(rows):
        for m in re.finditer(r"\d+", lines[row]):
            get_gear_neighbors(lines, rows, cols, row, m, gears)

    return sum(prod(ratios) for ratios in gears.values() if len(ratios) == 2)


sum_ratios(schema_test), 467835

(467835, 467835)

In [21]:
with open("../input/day3.txt") as f:
    print(sum_ratios(f.read()))

77509019


<link href="style.css" rel="stylesheet"></link>

<main>
<article><p>That's the right answer!  You are <span class="day-success">one gold star</span> closer to restoring snow operations.</p><p>You have completed Day 3! You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+%22Gear+Ratios%22+%2D+Day+3+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F3&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+%22Gear+Ratios%22+%2D+Day+3+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F3';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this victory or <a href="/2023">[Return to Your Advent Calendar]</a>.</p></article>
</main>
